# Train All 6 Classification Models
This notebook trains Logistic Regression, Decision Tree, KNN, Naive Bayes, Random Forest, and XGBoost on the Breast Cancer dataset.

In [ ]:
import json
from pathlib import Path

import joblib
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    matthews_corrcoef,
    precision_score,
    recall_score,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


RANDOM_STATE = 42
TEST_SIZE = 0.2
TARGET_COLUMN = "target"

In [ ]:
def make_models():
    return {
        "Logistic Regression": Pipeline(
            steps=[
                ("scaler", StandardScaler()),
                ("model", LogisticRegression(max_iter=2000, random_state=RANDOM_STATE)),
            ]
        ),
        "Decision Tree": DecisionTreeClassifier(random_state=RANDOM_STATE),
        "KNN": Pipeline(
            steps=[
                ("scaler", StandardScaler()),
                ("model", KNeighborsClassifier(n_neighbors=5)),
            ]
        ),
        "Naive Bayes": GaussianNB(),
        "Random Forest (Ensemble)": RandomForestClassifier(
            n_estimators=300, random_state=RANDOM_STATE
        ),
        "XGBoost (Ensemble)": XGBClassifier(
            n_estimators=300,
            learning_rate=0.05,
            max_depth=4,
            subsample=0.9,
            colsample_bytree=0.9,
            objective="binary:logistic",
            eval_metric="logloss",
            random_state=RANDOM_STATE,
        ),
    }


def evaluate_model(model, x_test, y_test):
    preds = model.predict(x_test)
    if hasattr(model, "predict_proba"):
        probs = model.predict_proba(x_test)[:, 1]
    else:
        probs = preds
    return {
        "Accuracy": accuracy_score(y_test, preds),
        "AUC": roc_auc_score(y_test, probs),
        "Precision": precision_score(y_test, preds),
        "Recall": recall_score(y_test, preds),
        "F1 Score": f1_score(y_test, preds),
        "MCC": matthews_corrcoef(y_test, preds),
    }

In [ ]:
try:
    base_dir = Path(__file__).resolve().parent
except NameError:
    base_dir = Path.cwd()

output_dir = base_dir
output_dir.mkdir(parents=True, exist_ok=True)

dataset = load_breast_cancer(as_frame=True)
df = dataset.frame.copy()
df.rename(columns={"target": TARGET_COLUMN}, inplace=True)
feature_names = [col for col in df.columns if col != TARGET_COLUMN]

x = df[feature_names]
y = df[TARGET_COLUMN]

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)

print(f"Train size: {len(x_train)}, Test size: {len(x_test)}")

In [ ]:
models = make_models()
metrics = {}

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(x_train, y_train)
    metrics[name] = evaluate_model(model, x_test, y_test)
    safe_name = name.lower().replace(" ", "_").replace("(", "").replace(")", "")
    joblib.dump(model, output_dir / f"{safe_name}.joblib")

print("\nAll models trained successfully!")

In [ ]:
metrics_path = output_dir / "metrics.json"
with metrics_path.open("w", encoding="utf-8") as f:
    json.dump(metrics, f, indent=2)

test_data = x_test.copy()
test_data[TARGET_COLUMN] = y_test
test_data.to_csv(output_dir / "test_data.csv", index=False)

with (output_dir / "feature_names.json").open("w", encoding="utf-8") as f:
    json.dump({"features": feature_names, "target": TARGET_COLUMN}, f, indent=2)

print("Saved models and metrics to:", output_dir)
pd.DataFrame(metrics).T.round(4)